<a href="https://colab.research.google.com/github/w1767/w1767/blob/main/NBeats_Forecasting_VVIX_VIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#transformers for time series data
# n-beats: neural basis expansion analysis for interpretable time series forecasting
# detrend data
#remove seasonality
#turn into supervised learning process
#get baseline, important for ML problems start with simple naive baseline = predict the last time step as the next step
#being able to quantify our confidence in forecasting referred to as uncertainty, confidence interval, prediction interval
#UBER uses Bayesian or MC dropout
# one way to do this is Monte Carlo dropout is to use dropout at inference and run multiple simulations
#two types of uncertainty
#1. coconut uncertainty - get hit on a beach by coconut = epistemic uncertainty/hard to estimate
#2. subway uncertainty - train should appear at 5pm but it shows up 1-2 min either side= aleatory uncertainty
#aleatory uncertainty = data uncertainty
#weather = predict every day 
#nate silver vs Nasim Taleb
# politic outcomes = open system, many different variables
#silver says that last outcome is all that matters
#taleb says tail risk
#WRMSSE = WEIGHT ROOT MEAN SQUARE ERROR
#kaggle 101 = GOT A TABULAR DATA? THEN STACK TOGETHER ENSEMBLES OF LIGHTGBM/XGBOOST
#replicate and modify https://arxiv.org/pdf/1905.10437.pdf 

In [3]:
!wget https://github.com/ElementAI/N-BEATS

--2022-04-20 12:57:00--  https://github.com/ElementAI/N-BEATS
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘N-BEATS’

N-BEATS                 [ <=>                ] 179.96K  --.-KB/s    in 0.1s    

2022-04-20 12:57:01 (1.47 MB/s) - ‘N-BEATS’ saved [184275]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = "/content/drive/MyDrive/PhD research/VVIX AND VIX DATA.xlsx"

In [1]:
#check for gpu
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-2d364408-4a53-7522-0143-5a3060ee1272)


In [11]:
#import pandas
import pandas as pd
#because data has date component, pandas will parse the dates using the parse_dates parameters
df=pd.read_csv("/content/drive/MyDrive/PhD research/VIX daily.csv",
                  parse_dates=["Date"],
                  index_col=["Date"]) # parse the date column and tell pandas that column 1 is a datetime
df.head()

,Last Price,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
Date,,,,,,,
2022-04-20,21.19,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-19,21.37,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-18,22.17,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-14,22.70,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-13,21.82,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df

,Last Price,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
Date,,,,,,,
2022-04-20,21.19,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-19,21.37,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-18,22.17,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-14,22.70,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-13,21.82,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12247 entries, 2022-04-20 to NaT
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Last Price  8136 non-null   float64
 1   Unnamed: 2  0 non-null      float64
 2   Unnamed: 3  0 non-null      float64
 3   Unnamed: 4  0 non-null      float64
 4   Unnamed: 5  0 non-null      float64
 5   Unnamed: 6  0 non-null      float64
 6   Unnamed: 7  0 non-null      float64
dtypes: float64(7)
memory usage: 765.4 KB


In [15]:
len(df)

12247

In [17]:
# only last price for each date
VIX_daily = pd.DataFrame(df["Last Price"]).rename(columns={"Last Price": "Price"})
VIX_daily.head()

,Price
Date,
2022-04-20,21.19
2022-04-19,21.37
2022-04-18,22.17
2022-04-14,22.70
2022-04-13,21.82


In [ ]:
#Plot
import matplotlib.pyplot as plt
VIX